In [ ]:
# default_exp modules

In [ ]:
#export
import collections

from abc import ABCMeta
from dataclasses import dataclass, field
from functools import partial

from traitlets import HasTraits, Dict, Unicode, All, Any
from abc import ABCMeta, abstractmethod
from io import StringIO
import typing
from typing import Optional, Mapping, Iterable, Tuple, Union, TYPE_CHECKING, List, Set
import traittypes
from IPython.display import display
from ipywidgets import FileUpload, Output, Widget
import pandas as pd

if TYPE_CHECKING:
    from dharpa_toolbox.workflows import DharpaWorkflow

@dataclass
class DisplayItem():
    name: str
    display_object: Widget
    metadata: Mapping[str, typing.Any] = field(default_factory=dict)

    def __repr__(self):

        return f"{self.__class__.__name__}(name={self.name} display_type={self.display_object.__class__.__name__} metadata={self.metadata})"


class DharpaModule(metaclass=ABCMeta):

    def __init__(self, id: str, input_mapping: Mapping[str, Union[str, Tuple[str, str]]]=None, config:Mapping[str, typing.Any]=None):

        self._id: str = id

        if config is None:
            config = {}
        self._config: Mapping[str, typing.Any] = config

        self._input_mapping: Mapping[str, Tuple[str, str]] = {}
        if input_mapping:
            for k, v in input_mapping.items():
                if isinstance(v, str):
                    self._input_mapping[k] = (v, k)
                elif isinstance(v, collections.Iterable) and len(v) == 2:
                    self._input_mapping[k] = (v[0], v[1])
                else:
                    raise ValueError(f"Invalid value for input reference: {v} -- must be a string or 2-item iterable")
        self._inputs: HasTraits = self._create_inputs()
        self._inputs.observe(self._input_updated, names=All)
        self._outputs: HasTraits = self._create_outputs()

        self._input_staging: Mapping[str, typing.Any] = {}
        self._stale_state: bool = True

    def _input_updated(self, change) -> typing.Any:

        # print(f"Input updated ({self.id}): {change}")

        if change.name not in self._input_staging.keys():
            self._input_staging[change.name] = {"old": change.old, "new": change.new}
        else:
            self._input_staging[change.name]["new"] = change.new
        self._stale_state = True

    def process(self, change):

        result = None

        # if self._stale_state:
        result = self._process(change)

        # print("RESULT")
        # print(result)

        self._input_staging.clear()
        if result:
            for k, v in result.items():
                self.outputs.set_trait(k, v)

        self._stale_state = False


    @abstractmethod
    def _process(self, change) -> Mapping[str, typing.Any]:
        pass

    @property
    def id(self) -> str:
        return self._id

    @property
    def stale(self) -> bool:
        return self._stale_state

    @property
    def inputs(self):
        return self._inputs

    @property
    def outputs(self):
        return self._outputs

    @property
    def input_mapping(self) -> Mapping[str, Tuple[str, str]]:
        return self._input_mapping

    @abstractmethod
    def _create_inputs(self) -> HasTraits:
        pass

    @abstractmethod
    def _create_outputs(self) -> HasTraits:
        pass

    def __repr__(self):

        return f"{self.__class__.__name__}(input_names={self.inputs.trait_names()} output_names={self.outputs.trait_names()})"


class EmptyObject(HasTraits):

    pass

class DataDisplayModule(DharpaModule):

    def _create_inputs(self) -> HasTraits:

        class DataValue(HasTraits):

            data = Dict(allow_none=True, default_value=None)

        return DataValue()

    def _create_outputs(self) -> HasTraits:

        return EmptyObject()

    def _process(self, change) -> Mapping[str, typing.Any]:

        return {"data": {"new": "data display"}}


class InputFilesModule(DharpaModule):

    def _process(self, change) -> Mapping[str, typing.Any]:

        return {"file_details": change.new}

    def _create_inputs(self) -> HasTraits:

        class InputFiles(HasTraits):

            data = Any(allow_none=True, default_value=None)

        return InputFiles()

    def _create_outputs(self) -> HasTraits:

        class InputFileValue(HasTraits):

            file_details = Dict()
            # content_data = traittypes.DataFrame()

        return InputFileValue()

class DharpaInputWidget(metaclass=ABCMeta):

    def __init__(self, module: DharpaModule):

        self._widget: Widget = self._create_widget(module)


    @abstractmethod
    def _create_widget(self, module: DharpaModule) -> Widget:
        pass

    @property
    def widget(self) -> Widget:
        return self._widget

class DharpaOutputWidget(metaclass=ABCMeta):

    def __init__(self, module: DharpaModule):

        self._widget: Widget = self._create_widget(module)


    @abstractmethod
    def _create_widget(self, module: DharpaModule) -> Widget:
        pass

    @property
    def widget(self) -> Widget:
        return self._widget

class InputFilesWidget(DharpaInputWidget):

    def _create_widget(self, module: DharpaModule) -> Widget:

        # uploaded_file_details = Output()
        file_upload = FileUpload(accept=".csv", multiple=False)

        def set_module_value(change):

            module.inputs.data = change.new
            # file_upload.value.clear()
            file_upload._counter = 0
            module.process(change)

        file_upload.observe(set_module_value, names="value")
        return file_upload

class DataDisplayWidget(DharpaOutputWidget):

    def _create_widget(self, module: DharpaModule) -> Widget:

        output = Output()

        def file_data_updated(change):

            output.clear_output()
            with output:
                print("XXXXXXXXXXXXXXXXX")
                print(change.new)

        module.outputs.observe(file_data_updated, names="file_details")
        return output


In [ ]:
from dharpa_toolbox.core import export_notebooks
export_notebooks()

Converted 00_core.ipynb.
Converted 01_modules.ipynb.
Converted 02_workflows.ipynb.
